In [101]:
import os
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.test.utils import datapath

model_file = "zh_word2vec.model"
model = Word2Vec.load(model_file)

model.wv.most_similar("说", topn=30)



[('为什么', 0.8298794627189636),
 ('问', 0.8270461559295654),
 ('说道', 0.8173126578330994),
 ('为何', 0.8128331899642944),
 ('告诉', 0.7882134318351746),
 ('吗', 0.7878919839859009),
 ('的话', 0.7855218648910522),
 ('所说', 0.7767419219017029),
 ('觉得', 0.7696527242660522),
 ('讲', 0.7678855657577515),
 ('怎么样', 0.7658766508102417),
 ('声称', 0.764196515083313),
 ('以为', 0.7633250951766968),
 ('问道', 0.7630106210708618),
 ('认为', 0.758924126625061),
 ('中说', 0.751205325126648),
 ('时说', 0.7498580813407898),
 ('写道', 0.7494862079620361),
 ('告诫', 0.7469834089279175),
 ('说起', 0.7460322380065918),
 ('确信', 0.7414821982383728),
 ('质问', 0.7364925146102905),
 ('谈论', 0.7361835241317749),
 ('感叹', 0.7359782457351685),
 ('说出', 0.7354288101196289),
 ('谁', 0.7353894710540771),
 ('称', 0.7324063181877136),
 ('却说', 0.7310951948165894),
 ('也许', 0.7306363582611084),
 ('或许', 0.730185866355896)]

In [25]:
import pandas as pd

In [26]:
news_chinese = pd.read_csv("/home/weiliang/news_chinese_201908201131.csv")

In [4]:
news = news_chinese.content.to_list()

In [116]:
import re 
import jieba
def clean(news_text):
    text_only = ''.join(re.findall(r'\w+', news_text)) # only retain valid words
    return text_only.replace('n', '') # remove n in news
def cut(string):
    return list(jieba.cut(string))

In [117]:
news_clean = [' '.join(cut(clean(n))) for n in news]
news_news_clean_len = len(news_clean)
print(news_news_clean_len)


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.030 seconds.
Prefix dict has been built succesfully.


70173


In [118]:
word_total = sum([len(nc.split()) for nc in news_clean])
print(word_total)

11877241


In [119]:
with open('chinese_news.corps', 'a+') as f:
    for nc in news_clean:
        f.write(nc + "\n")

In [120]:
sentences = LineSentence('chinese_news.corps')

In [121]:
model1 = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)

In [125]:
model1.wv.most_similar('说', topn=20)

[('表示', 0.8463233709335327),
 ('认为', 0.7879735231399536),
 ('指出', 0.7721938490867615),
 ('告诉', 0.7540926337242126),
 ('说完', 0.715735673904419),
 ('看来', 0.6958412528038025),
 ('称', 0.6870326995849609),
 ('介绍', 0.6813185214996338),
 ('坦言', 0.6690934896469116),
 ('透露', 0.6331932544708252),
 ('中说', 0.6209747791290283),
 ('明说', 0.584187388420105),
 ('罗恩费伦', 0.5742655992507935),
 ('所说', 0.5712404251098633),
 ('强调', 0.5711138248443604),
 ('时说', 0.5316095948219299),
 ('建议', 0.5299563407897949),
 ('文说', 0.5233355760574341),
 ('写道', 0.4978210926055908),
 ('温德姆', 0.49491986632347107)]

### train model with chinsese news

In [132]:
model.epochs

5

In [137]:

model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(49138219, 59386205)

In [138]:
model.save("wiki_and_chinese_news_word2vec.model")

In [139]:
model.wv.most_similar("说", topn=30)

[('表示', 0.8655452132225037),
 ('告诉', 0.8081321120262146),
 ('认为', 0.8065691590309143),
 ('指出', 0.7971088886260986),
 ('说完', 0.7543131113052368),
 ('介绍', 0.6950348615646362),
 ('坦言', 0.6608978509902954),
 ('称', 0.6517000198364258),
 ('看来', 0.6478255987167358),
 ('时说', 0.6423693895339966),
 ('强调', 0.6018719673156738),
 ('中说', 0.5731960535049438),
 ('透露', 0.5685068964958191),
 ('相信', 0.5671338438987732),
 ('所说', 0.5622093677520752),
 ('深信', 0.5584651231765747),
 ('并不认为', 0.5563322305679321),
 ('的话', 0.5453881025314331),
 ('写道', 0.5218533277511597),
 ('形容', 0.5168880224227905),
 ('坚信', 0.5134227871894836),
 ('還說', 0.50819993019104),
 ('認为', 0.5064188241958618),
 ('明白', 0.5037347078323364),
 ('認為', 0.5025042295455933),
 ('确信', 0.5017237663269043),
 ('深知', 0.5001986622810364),
 ('建议', 0.4999033808708191),
 ('说道', 0.49879705905914307),
 ('问', 0.496876060962677)]

In [148]:
from collections import defaultdict
def most_similar(start_word, max_layer=3):
    word_paths = [[start_word]]
    occurences = defaultdict(int)
    searched_words = {}
    similar_words_in_layers = [start_word]
    while True:
        if len(word_paths) > 0:
            word_path = word_paths.pop(0)
        else:
            return occurences
        word = word_path[-1]
        if word in searched_words:
            similar_words = searched_words[word]
            
        else:
            similar_words_with_possibility = model.wv.most_similar(word, topn=30)
            similar_words = [word for word, posibility in similar_words_with_possibility]
        for similar_word in similar_words:
            if start_word == similar_word:
                if word not in similar_words_in_layers:
                    #similar_words_in_layers.append(word)
                    for word in word_path:
                        occurences[word] += 1
            else:
                new_word_path = word_path+[similar_word]
                if similar_word not in word_path and len(new_word_path) <= max_layer:
                    word_paths.append(new_word_path)
                
        searched_words[word] = similar_words


In [149]:
most_similar_words = most_similar("说", 3)
from collections import Counter
counter = Counter(most_similar_words)

In [150]:
counter.most_common(20)

[('说', 171),
 ('认为', 30),
 ('表示', 28),
 ('告诉', 27),
 ('指出', 27),
 ('看来', 22),
 ('说完', 21),
 ('时说', 20),
 ('坦言', 19),
 ('强调', 17),
 ('中说', 17),
 ('介绍', 16),
 ('称', 16),
 ('透露', 15),
 ('所说', 12),
 ('相信', 10),
 ('建议', 10),
 ('的话', 5),
 ('形容', 2),
 ('写道', 2)]

In [145]:
model.wv.most_similar("说", topn=20)

[('表示', 0.8655452132225037),
 ('告诉', 0.8081321120262146),
 ('认为', 0.8065691590309143),
 ('指出', 0.7971088886260986),
 ('说完', 0.7543131113052368),
 ('介绍', 0.6950348615646362),
 ('坦言', 0.6608978509902954),
 ('称', 0.6517000198364258),
 ('看来', 0.6478255987167358),
 ('时说', 0.6423693895339966),
 ('强调', 0.6018719673156738),
 ('中说', 0.5731960535049438),
 ('透露', 0.5685068964958191),
 ('相信', 0.5671338438987732),
 ('所说', 0.5622093677520752),
 ('深信', 0.5584651231765747),
 ('并不认为', 0.5563322305679321),
 ('的话', 0.5453881025314331),
 ('写道', 0.5218533277511597),
 ('形容', 0.5168880224227905)]

## Use standford CoreNLP to parse news

In [90]:
import sys
!{sys.executable} -m pip install stanford-corenlp

Looking in indexes: https://pypi.douban.com/simple


In [91]:
import os 
os.environ["CORENLP_HOME"] = "/Users/weiliang/Code/AI/nlp/project1/stanford-corenlp-full-2018-10-05/"

In [92]:
import corenlp

text = "奥巴马说这个种事情怎么可以浪费这么多时间。这是什么情况。"

# We assume that you've downloaded Stanford CoreNLP and defined an environment
# variable $CORENLP_HOME that points to the unzipped directory.
# The code below will launch StanfordCoreNLPServer in the background
# and communicate with the server to annotate the sentence.
with corenlp.CoreNLPClient(start_server=True,annotators="tokenize ssplit ner depparse".split(), output_format="json") as client:
  ann = client.annotate(text)

# You can access annotations using ann.
#sentence = ann.sentence[0]
print(ann['sentences'][1])

{'index': 1, 'basicDependencies': [{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 4, 'dependentGloss': '情况'}, {'dep': 'nsubj', 'governor': 4, 'governorGloss': '情况', 'dependent': 1, 'dependentGloss': '这'}, {'dep': 'cop', 'governor': 4, 'governorGloss': '情况', 'dependent': 2, 'dependentGloss': '是'}, {'dep': 'det', 'governor': 4, 'governorGloss': '情况', 'dependent': 3, 'dependentGloss': '什么'}, {'dep': 'punct', 'governor': 4, 'governorGloss': '情况', 'dependent': 5, 'dependentGloss': '。'}], 'enhancedDependencies': [{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 4, 'dependentGloss': '情况'}, {'dep': 'nsubj', 'governor': 4, 'governorGloss': '情况', 'dependent': 1, 'dependentGloss': '这'}, {'dep': 'cop', 'governor': 4, 'governorGloss': '情况', 'dependent': 2, 'dependentGloss': '是'}, {'dep': 'det', 'governor': 4, 'governorGloss': '情况', 'dependent': 3, 'dependentGloss': '什么'}, {'dep': 'punct', 'governor': 4, 'governorGloss': '情况', 'dependent': 5, 'dependentGloss

In [94]:
# coding: utf8
print(ann['sentences'][0])

{'index': 0, 'basicDependencies': [{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 2, 'dependentGloss': '说'}, {'dep': 'nsubj', 'governor': 2, 'governorGloss': '说', 'dependent': 1, 'dependentGloss': '奥巴马'}, {'dep': 'det', 'governor': 5, 'governorGloss': '事情', 'dependent': 3, 'dependentGloss': '这个'}, {'dep': 'mark:clf', 'governor': 3, 'governorGloss': '这个', 'dependent': 4, 'dependentGloss': '种'}, {'dep': 'nsubj', 'governor': 8, 'governorGloss': '浪费', 'dependent': 5, 'dependentGloss': '事情'}, {'dep': 'advmod', 'governor': 8, 'governorGloss': '浪费', 'dependent': 6, 'dependentGloss': '怎么'}, {'dep': 'aux:modal', 'governor': 8, 'governorGloss': '浪费', 'dependent': 7, 'dependentGloss': '可以'}, {'dep': 'ccomp', 'governor': 2, 'governorGloss': '说', 'dependent': 8, 'dependentGloss': '浪费'}, {'dep': 'advmod', 'governor': 10, 'governorGloss': '多', 'dependent': 9, 'dependentGloss': '这么'}, {'dep': 'amod', 'governor': 11, 'governorGloss': '时间', 'dependent': 10, 'dependentGloss': '多'}, 

TypeError: write() argument must be str, not Sentence